In [1]:
# @name graph_v3.2_v20190616
# @description notebook to build the NGLY1 Deficiency review knowledge graph v3.2
# @author Núria Queralt Rosinach
# @date 16 June 2019

# Description

This is the notebook for the creation of the first review network and derived hypotheses. 

* Using intermediary variables from workflow objects. In this workflow variables are directly used for the next step. 


* Review network: From Monarch knowledge graph, we built a network seeded by 8 nodes, retrieving their explicit relationships and all the relationships among all these nodes. Seed nodes:

    - 'MONDO:0007739' HD
    - 'HGNC:4851' Htt
    - 'CHEBI:18248' Iron (not working)
    - 'HGNC:18229' Rhes (RASD2)
    
Possible seed nodes:
https://monarchinitiative.org/search/Iron
* Connecting paths: query templates.

In [1]:
import transcriptomics, regulation, curation, monarch, graph, neo4jlib, hypothesis, summary, utils
import pandas as pd

In [ ]:
#onarch.get_connections()

In [ ]:
#nodes, edges = monarch.get_neighbours(["HGNC:18229"])

In [ ]:
#len(nodes)

In [ ]:
#nodes

In [ ]:
#nodes2, edges2 = monarch.get_neighbours(nodes)

## Edges library
### Review edges to integrate into the knowledge graph and prepare them as individual networks

#### TRANSCRIPTOMICS NETWORK
#### import transcriptomics
We retrieved edges from RNA-seq transcriptomics profiles using the `transcriptomics` module:

    - Experimental data sets: from Chow et al. paper [pmid:29346549] (NGLY1 deficiency model on fruit fly)

In [2]:
%%time
# prepare data to graph schema
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path, "\t")


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().

CPU times: user 45.9 ms, sys: 2.37 ms, total: 48.3 ms
Wall time: 46.9 ms


In [7]:
%%time
# prepare data to graph schema
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path, "\t")
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes, nodes_dict = transcriptomics.build_nodes(rna_network)
rna_edges = transcriptomics.rework_edges(pd.DataFrame(rna_edges), nodes_dict)


The function "read_data()" is running...

* This is the size of the raw expression data structure: (28087, 10)
* These are the expression attributes: Index(['Unnamed: 0', 'symbol', 'baseMean', 'HD.mean', 'Control.mean',
       'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj'],
      dtype='object')
* This is the first record:
           Unnamed: 0 symbol  baseMean    HD.mean  Control.mean  \
0  ENSG00000069011.10  PITX1  5.645675  18.684286      0.323793   

   log2FoldChange     lfcSE      stat        pvalue          padj  
0        4.769658  0.366367  13.01879  9.567529e-39  2.687232e-34  

The raw data is saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/transcriptomics/HD/data/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.csv


Finished read_data().


The function "clean_data()" is running. Keeping only data with FC > 1.5 and FDR < 5% ...

* This is the size of the clean expression data structure: (3209, 6)
* These are the clean expressi

In [9]:
df = pd.DataFrame(rna_edges)

In [11]:
df.to_csv("rna_edges_rework.csv")

In [94]:
test = transcriptomics.rework_edges(pd.DataFrame(rna_edges), nodes_dict)


* This is the size of the edges file data structure: (3209, 9)
* These are the edges attributes: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
* This is the first record:
  subject_id  object_id property_id  property_label property_description  \
0  HGNC:4851  HGNC:9004  RO:0002434  interacts with                   NA   

                                property_uri  \
0  http://purl.obolibrary.org/obo/RO_0002434   

                                  reference_uri  \
0  https://www.ncbi.nlm.nih.gov/pubmed/26636579   

                           reference_supporting_text reference_date  
0  Here we present a genome-wide analysis of mRNA...     2015-11-04  

The transcriptomics network edges are built and saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/graph/rna_edges_v2021-04-07.csv


Finished bui

,subject_id,object_id,property_id,property_label,property_description,property_uri,reference_uri,reference_supporting_text,reference_date
0,HGNC:4851,HGNC:9004,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
1,HGNC:4851,HGNC:5120,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
2,HGNC:4851,HGNC:5122,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
3,HGNC:4851,HGNC:5101,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
4,HGNC:4851,HGNC:5100,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
...,...,...,...,...,...,...,...,...,...
3204,HGNC:4851,HGNC:55493,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
3205,HGNC:4851,ensembl:ENSG00000181227,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
3206,HGNC:4851,HGNC:53465,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04
3207,HGNC:4851,HGNC:15722,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/26636579,Here we present a genome-wide analysis of mRNA...,2015-11-04


- Transcriptomics network is returned as both digital object (`rna_edges`, `rna_nodes`) and CSV files at _**graph/**_ (`rna_edges_version.csv`, `rna_nodes_version.csv`)

In [5]:
# print type of objects
print('type edges:', type(rna_edges))
print('type nodes:', type(rna_nodes))
print()

# print objects sizes
print('len edges:', len(rna_edges))
print('len nodes:', len(rna_nodes))
print()

# print object attribute
print('attribute edges:', rna_edges[0].keys())
print('attribute nodes:', rna_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 3209
len nodes: 3210

attribute edges: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### REGULATION NETWORK
#### import regulation

We retrieved human TF gene expression regulation edges from several sources using the `regulation` module.

In [6]:
%%time
# prepare msigdb data
gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)


The function "prepare_msigdb_data()" is running...

* Number of Transcription Factor Targets (TFT) gene sets: 615

The MSigDB raw network is saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/regulation/msigdb/out/tf_genelist_entrez_msigdb.json. Other reporting files are also saved at the same directory.


Finished prepare_msigdb_data().


The function "load_tf_gene_edges()" is running...

Finished load_tf_gene_edges().


The function "get_gene_id_normalization_dictionaries()" is running...

* Querying BioThings to map gene symbols to HGNC and Entrez IDs...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
48 input query terms found no hit:
	['E2F1DP1', 'ALPHACP1', 'E2F4DP1', 'COREBINDINGFACTOR', 'E2F4DP2', 'CEBPGAMMA', 'CDPCR3', 'AMEF2', '
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/LUMC/HDSR/bioknowledge-

- Regulation network is returned as both digital object (`reg_edges`, `reg_nodes`) and CSV files at _**graph/**_ (`regulation_edges_version.csv`, `regulation_nodes_version.csv`)

In [81]:
reg_edges

,subject_id,object_id,property_id,property_label,property_description,property_uri,reference_uri,reference_supporting_text,reference_date
0,HGNC:8615,HGNC:8803,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/17202159,"This edge comes from the TRED dataset in ""tfta...",2007-01-01
1,HGNC:8615,HGNC:12687,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/17202159,"This edge comes from the TRED dataset in ""tfta...",2007-01-01
2,HGNC:8615,HGNC:10970,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/17202159,"This edge comes from the TRED dataset in ""tfta...",2007-01-01
3,HGNC:8615,HGNC:10972,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/17202159,"This edge comes from the TRED dataset in ""tfta...",2007-01-01
4,HGNC:8615,HGNC:25567,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,https://www.ncbi.nlm.nih.gov/pubmed/17202159,"This edge comes from the TRED dataset in ""tfta...",2007-01-01
...,...,...,...,...,...,...,...,...,...
197262,HGNC:16764,HGNC:10931,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,http://www.broadinstitute.org/gsea/msigdb/card...,"This edge comes from the C3:TFT dataset in ""ms...",NaN
197263,HGNC:16764,HGNC:24561,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,http://www.broadinstitute.org/gsea/msigdb/card...,"This edge comes from the C3:TFT dataset in ""ms...",NaN
197264,HGNC:16764,HGNC:11209,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,http://www.broadinstitute.org/gsea/msigdb/card...,"This edge comes from the C3:TFT dataset in ""ms...",NaN
197265,HGNC:16764,HGNC:26314,RO:0002434,interacts with,NA,http://purl.obolibrary.org/obo/RO_0002434,http://www.broadinstitute.org/gsea/msigdb/card...,"This edge comes from the C3:TFT dataset in ""ms...",NaN


In [7]:
# print type of objects
print('type edges:', type(reg_edges))
print('type nodes:', type(reg_nodes))
print()

# print objects sizes
print('len edges:', len(reg_edges))
print('len nodes:', len(reg_nodes))
print()

# print object attribute
print('attribute edges:', reg_edges[0].keys())
print('attribute nodes:', reg_nodes[0].keys())

type edges: <class 'list'>
type nodes: <class 'list'>

len edges: 197267
len nodes: 16968

attribute edges: dict_keys(['subject_id', 'object_id', 'property_id', 'property_label', 'property_description', 'property_uri', 'reference_uri', 'reference_supporting_text', 'reference_date'])
attribute nodes: dict_keys(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms', 'description'])


#### CURATED NETWORK
#### import curation

We retrieved and prepared curated edges using the `curation` module. 

In [8]:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")

In [9]:
%%time
# graph v3.2
# read network from drive and concat all curated statements
curation_edges, curation_nodes = curation.read_network(version='v20180118')

# prepare data edges and nodes
data_edges = curation.prepare_data_edges(curation_edges)
data_nodes = curation.prepare_data_nodes(curation_nodes)

# prepare curated edges and nodes
curated_network = curation.prepare_curated_edges(data_edges)
curated_concepts = curation.prepare_curated_nodes(data_nodes)


# build edges and nodes files
curation_edges = curation.build_edges(curated_network)
curation_nodes = curation.build_nodes(curated_concepts)


The function "read_network()" is running...

Reading and concatenating all curated statements in the network...

* Curation edge files concatenated shape: (322, 22)

Reading and concatenating all curated nodes in the network...

* Curation node files concatenated shape: (318, 9)

Finished read_network().


The function "prepare_data_edges()" is running...

Preparing curated network...

Saving curated network at curation/...

*Curated edges data structure shape: (321, 9)
*Curated edges data structure fields: Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')

The curated edges are saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/curation/curated_edges_v2021-04-07.csv


Finished prepare_data_edges().


The function "prepare_data_nodes()" is running...

Preparing curated nodes...

Saving curated nodes at 

KeyError: 'property_label'

- Curated network is returned as both digital object (`curation_edges`, `curation_nodes`) and CSV files at _**graph/**_ (`curated_graph_edges_version.csv`, `curated_graph_nodes_version.csv`)
- The original curated network, i.e. without graph data model normalization, is saved as CSV files at _**curation/**_ (`curated_edges_version.csv`, `curated_nodes_version.csv`)

In [10]:
# print type of objects
print('type edges:', type(curation_edges))
print('type nodes:', type(curation_nodes))
print()

# print objects sizes
print('len edges:', len(curation_edges))
print('len nodes:', len(curation_nodes))
print()

# print object attribute
print('attribute edges:', curation_edges[0].keys())
print('attribute nodes:', curation_nodes[0].keys())

type edges: <class 'pandas.core.frame.DataFrame'>
type nodes: <class 'pandas.core.frame.DataFrame'>

len edges: 322
len nodes: 318



KeyError: 0

#### MONARCH NETWORK
#### import monarch
We retrieved edges from Monarch using the `monarch` module.

Tasks:

- From 8 seed nodes we retrieved 1st shell nodes
- From all seed and 1st shell nodes we retrieved ortho-phenotypes
- We retrieved extra edges among all of them, i.e. extra connectivity between: seed, 1st shell, ortholog-phenotype nodes

In [ ]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
    'REACT:R-HSA-917937' #Iron uptake pathway
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))
# get second degree of neighbours
monarch_network = monarch.get_connections(neighboursList)
print(len(monarch_network))

In [ ]:
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

In [ ]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

In [ ]:
%%time
# prepare data to graph schema
# seed nodes
seedList = [ 
    'MONDO:0007739', # HD
    'HGNC:4851', # Htt
    'HGNC:182293', # Rhes
] 

# get first shell of neighbours
neighboursList = monarch.get_neighbours_list(seedList)
print(len(neighboursList))

# introduce animal model ortho-phenotypes for seed and 1st shell neighbors
## For seed nodes:
seed_orthophenoList = monarch.get_orthopheno_list(seedList)
print(len(seed_orthophenoList))
## For 1st shell nodes:
neighbours_orthophenoList = monarch.get_orthopheno_list(neighboursList)
print(len(neighbours_orthophenoList))

# network nodes: seed + 1shell + ortholog-phentoype
geneList = sum([seedList,
                neighboursList,
                #seed_orthophenoList,
                #neighbours_orthophenoList
               ], 
               [])
print('genelist: ',len(geneList))

# get Monarch network
monarch_network = monarch.extract_edges(geneList)
print('network: ',len(monarch_network))

# save edges
monarch.print_network(monarch_network, 'monarch_connections')

# build network with graph schema 
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

In [ ]:
monarch_network = monarch.read_connections("monarch_connections_v2020-11-15.csv")
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

In [ ]:
monarch_edges = monarch.build_edges(monarch_network)


In [ ]:
monarch_nodes = monarch.build_nodes(monarch_network)

In [ ]:
# print type of objects
print('type edges:', type(monarch_edges))
print('type nodes:', type(monarch_nodes))
print()

# print objects sizes
print('len edges:', len(monarch_edges))
print('len nodes:', len(monarch_nodes))
print()

# print object attribute
print('attribute edges:', monarch_edges[0].keys())
print('attribute nodes:', monarch_nodes[0].keys())

- Monarch network is returned as both digital object (`monarch_edges`, `monarch_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`)

## Graph library
### Create the review knowledge graph
#### import graph

Tasks:

* Load Networks and calculate graph nodes
* Retrieve extra connectivity for the graph from Monarch
* Build the review graph

In [ ]:
%%time
# load networks and calculate graph nodes
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)

# Monarch graph connectivity
## get Monarch edges
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))

## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')

## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

# build review graph
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)

In [ ]:
# obtain variables:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")

#monarch_network = monarch.read_connections("monarch_connections_v2020-11-15.csv")
#monarch_edges = monarch.build_edges(monarch_network)
#monarch_nodes = monarch.build_nodes(monarch_network)

#monarch_network_graph = monarch.read_connections("monarch_connections_graph_v2020-11-16.csv")
#monarch_graph_edges = monarch.build_edges(monarch_network_graph)
#monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path)
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes = transcriptomics.build_nodes(rna_network)

gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)

#graph_nodes_list, reg_graph_edges = graph.graph_nodes(
#    curation=curation_edges,
#    monarch=monarch_edges,
#    transcriptomics=rna_edges,
#    regulation=reg_edges
#)


In [ ]:
# fix curation data:
curation_nodes = curation_nodes.astype('object')
#curation_nodes.loc[0]

In [ ]:
curation_nodes['name'] = ["NaN", "NaN"]

In [ ]:
curation_nodes.loc[0]

In [ ]:
# construct df's
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)

In [ ]:
nodes = pd.read_csv("/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/graph/graph_nodes_v2020-11-20.csv")
edges = pd.read_csv("/home/karolis/Structured review/bioknowledge-reviewer/bioknowledge_reviewer/graph/graph_edges_v2020-11-20.csv")

- Regulation edges _merged_ with the graph is returned as both digital object (`reg_graph_edges`) and CSV file at _**graph/**_ (`regulation_graph_edges_version.csv`)
- Monarch network is returned as both digital object (`monarch_graph_edges`, `monarch_graph_nodes`) and CSV files at _**monarch/**_ (`monarch_edges_version.csv`, `monarch_nodes_version.csv`) overwritten the previous one.
- Review knowledge graph is returned as both digital object (`edges`, `nodes`) and CSV files at _**graph/**_ (`graph_edges_version.csv`, `graph_nodes_version.csv`)

In [ ]:
# print type of objects
print('type edges:', type(edges))
print('type nodes:', type(nodes))
print()

# print objects sizes
print('len edges:', len(edges))
print('len nodes:', len(nodes))
print()

# print object attribute
print('attribute edges:', edges.columns)
print('attribute nodes:', nodes.columns)

## Neo4jlib library
### Import the graph into Neo4j graph database
#### import neo4jlib

Tasks:

- Create Neo4j server instance
- Import review graph into the Neo4j graph database

In [16]:
curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")
curation_nodes = curation_nodes.astype('object')
curation_nodes['name'] = ["NaN", "NaN"]
curation_edges = curation_edges.to_dict("records")

In [17]:
# obtain variables:
#curation_edges = pd.read_csv("curation/data/HD/Empty_edges.csv")
#curation_nodes = pd.read_csv("curation/data/HD/Empty_nodes.csv")

monarch_network = monarch.read_connections("monarch_connections_v2020-12-12.csv")
monarch_edges = monarch.build_edges(monarch_network)
monarch_nodes = monarch.build_nodes(monarch_network)

#monarch_network_graph = monarch.read_connections("monarch_connections_graph_v2020-12-02.csv")
#monarch_graph_edges = monarch.build_edges(monarch_network_graph)
#monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

#csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
#data = transcriptomics.read_data(csv_path, "\t")
#clean_data = transcriptomics.clean_data(data)
#data_edges = transcriptomics.prepare_data_edges(clean_data)
#rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
#rna_edges = transcriptomics.build_edges(rna_network)
#rna_nodes, rna_edges = transcriptomics.build_nodes(rna_network)
csv_path = './transcriptomics/GSE64810_mlhd_DESeq2_diffexp_DESeq2_outlier_trimmed_adjust.txt'
data = transcriptomics.read_data(csv_path, "\t")
clean_data = transcriptomics.clean_data(data)
data_edges = transcriptomics.prepare_data_edges(clean_data)
rna_network = transcriptomics.prepare_rna_edges(data_edges)

# build network with graph schema
rna_edges = transcriptomics.build_edges(rna_network)
rna_nodes, nodes_dict = transcriptomics.build_nodes(rna_network)


gmt_path = './regulation/msigdb/data/c3.tft.v6.1.entrez.gmt'
regulation.prepare_msigdb_data(gmt_path)

# prepare individual networks
data = regulation.load_tf_gene_edges()
dicts = regulation.get_gene_id_normalization_dictionaries(data)
data_edges = regulation.prepare_data_edges(data, dicts)

# prepare regulation network
reg_network = regulation.prepare_regulation_edges(data_edges)

# build network with graph schema
reg_edges = regulation.build_edges(reg_network)
reg_nodes = regulation.build_nodes(reg_network)

#graph_nodes_list, reg_graph_edges = graph.graph_nodes(
#    curation=curation_edges,
#    monarch=monarch_edges,
#    transcriptomics=rna_edges,
#    regulation=reg_edges
#)



* This is the size of the data structure: (233182, 7)
* These are the attributes: Index(['object_id', 'object_label', 'reference_id_list', 'relation_id',
       'relation_label', 'subject_id', 'subject_label'],
      dtype='object')
* This is the first record:
    object_id object_label  reference_id_list    relation_id  \
0  MGI:892004        H2-Bl                NaN  RO:HOM0000017   

                   relation_label   subject_id subject_label  
0  in orthology relationship with  RGD:1595815      RT1-M6-2  

The function "build_edges()" is running...
df (233182, 9)

* This is the size of the edges file data structure: (233182, 9)
* These are the edges attributes: Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
* This is the first record:
    subject_id   object_id    property_id                  property_label  \
0  RGD:1595

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-3071...done.
Finished.
48 input query terms found no hit:
	['MMEF2', 'SIN3AK20', 'CETS1P54', 'CDPCR3', 'AHRARNT', 'ISRE', 'TAL1BETAE47', 'COMP1', 'CEBPGAMMA', 
Pass "returnall=True" to return complete lists of duplicate or missing query terms.

Saving not found gene symbols at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/regulation/not_found_symbols.list


* Querying BioThings to map Entrez to HGNC IDs and gene symbols...
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-1

In [18]:
curation_edges = pd.DataFrame(curation_edges)
monarch_edges = pd.DataFrame(monarch_edges)
rna_edges = pd.DataFrame(rna_edges)
curation_edges = curation_edges.iloc[0]
curation_edges = pd.DataFrame(curation_edges).T
reg_edges = pd.DataFrame(reg_edges)
curation_nodes = curation_nodes.iloc[0]
curation_nodes = pd.DataFrame(curation_nodes).T
monarch_nodes = pd.DataFrame(monarch_nodes)
rna_nodes = pd.DataFrame(rna_nodes)
reg_nodes = pd.DataFrame(reg_nodes)

[{'id': 'HGNC:8615',
  'semantic_groups': 'GENE',
  'preflabel': 'PAX1',
  'name': 'paired box 1',
  'synonyms': 'HUP48|OFC2',
  'description': 'This gene is a member of the paired box (PAX) family of transcription factors. Members of the PAX family typically contain a paired box domain and a paired-type homeodomain. These genes play critical roles during fetal development. This gene plays a role in pattern formation during embryogenesis and may be essential for development of the vertebral column. This gene is silenced by methylation in ovarian and cervical cancers and may be a tumor suppressor gene. Mutations in this gene are also associated with vertebral malformations. [provided by RefSeq, Mar 2012].'},
 {'id': 'HGNC:8803',
  'semantic_groups': 'GENE',
  'preflabel': 'PDGFRA',
  'name': 'platelet derived growth factor receptor alpha',
  'synonyms': 'CD140A|PDGFR-2|PDGFR2',
  'description': 'This gene encodes a cell surface tyrosine kinase receptor for members of the platelet-derive

In [8]:
monarch_nodes

In [9]:
curation_edges

,subject_id,property_id,object_id,reference_uri,reference_supporting_text,reference_date,property_label,property_description,property_uri
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
monarch_edges

,subject_id,object_id,property_id,property_label,property_description,property_uri,reference_uri,reference_supporting_text,reference_date
0,RGD:1595815,MGI:892004,RO:HOM0000017,in orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000017,NA,This edge comes from the Monarch Knowledge Gra...,NA
1,ENSEMBL:ENSRNOG00000061639,MGI:3704134,RO:HOM0000017,in orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000017,NA,This edge comes from the Monarch Knowledge Gra...,NA
2,FlyBase:FBgn0041191,MGI:97912,RO:HOM0000020,in 1 to 1 orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000020,NA,This edge comes from the Monarch Knowledge Gra...,NA
3,MGI:1344381,ENSEMBL:ENSOANG00000006132,RO:HOM0000020,in 1 to 1 orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000020,NA,This edge comes from the Monarch Knowledge Gra...,NA
4,ENSEMBL:ENSCAFG00000017647,ENSEMBL:ENSECAG00000019077,RO:HOM0000020,in 1 to 1 orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000020,NA,This edge comes from the Monarch Knowledge Gra...,NA
...,...,...,...,...,...,...,...,...,...
233177,RGD:1311415,ZFIN:ZDB-GENE-030131-7310,RO:HOM0000020,in 1 to 1 orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000020,NA,This edge comes from the Monarch Knowledge Gra...,NA
233178,dictyBase:DDB_G0271282,ZFIN:ZDB-GENE-030131-8680,RO:HOM0000017,in orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000017,NA,This edge comes from the Monarch Knowledge Gra...,NA
233179,ZFIN:ZDB-GENE-041008-78,UBERON:0000465,RO:0002206,expressed in,NA,http://purl.obolibrary.org/obo/RO_0002206,NA,This edge comes from the Monarch Knowledge Gra...,NA
233180,ENSEMBL:ENSECAG00000035314,MGI:95915,RO:HOM0000017,in orthology relationship with,NA,http://purl.obolibrary.org/obo/RO_HOM0000017,NA,This edge comes from the Monarch Knowledge Gra...,NA


In [ ]:
reg_edges

In [19]:
%%time
# load networks and calculate graph nodes
graph_nodes_list, reg_graph_edges = graph.graph_nodes(
    curation=curation_edges,
    monarch=monarch_edges,
    transcriptomics=rna_edges,
    regulation=reg_edges
)

# Monarch graph connectivity
## get Monarch edges
monarch_network_graph = monarch.extract_edges(graph_nodes_list)
print('network: ',len(monarch_network_graph))

## save Monarch network
monarch.print_network(monarch_network_graph, 'monarch_connections_graph')

## build Monarch network with graph schema
monarch_graph_edges = monarch.build_edges(monarch_network_graph)
monarch_graph_nodes = monarch.build_nodes(monarch_network_graph)

# build review graph
edges = graph.build_edges(
    curation=curation_edges,
    monarch=monarch_graph_edges,
    transcriptomics=rna_edges,
    regulation=reg_graph_edges
)
nodes = graph.build_nodes(
    statements=edges,
    curation=curation_nodes,
    monarch=monarch_graph_nodes,
    transcriptomics=rna_nodes,
    regulation=reg_nodes
)


The function "graph_nodes()" is running...

Preparing networks...
Curated:
(1, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')
Monarch:
(233182, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
Transcriptomics:
(3209, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')
Regulatory:
(197267, 9)
Index(['subject_id', 'object_id', 'property_id', 'property_label',
       'property_description', 'property_uri', 'reference_uri',
       'reference_supporting_text', 'reference_date'],
      dtype='object')

Concatenatin

  0%|          | 0/22997 [00:00<?, ?it/s]

(250556, 9)

Generating graph nodes...
(22997, 1)

Finished graph_nodes().


The function "extract_edges()" is running. Its runtime may take some hours. If you interrupt the process, you will lose all the edges retrieved and you should start over the execution of this function.


100%|██████████| 22997/22997 [29:01:03<00:00,  4.54s/it]   



Finished extract_edges(). To save the retrieved Monarch edges use the function "print_network()".

network:  575849

Saving Monarch edges at: '/home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_connections_graph_v2021-04-12.csv'...


The function "build_edges()" is running...
Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'
The edge that includes this new reference source is Pandas(Index=4010, subject_id='HGNC:7675', subject_label='NDN', relation_id='RO:0002200', relation_label='has phenotype', object_id='HP:0002808', object_label='Kyphosis', reference_id_list='NCBIBSgene:pws')
The method will continue to run without problem, writing the CURIE instead of the URI,until the dictionary is updated.
Detected a new

...

Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'
The edge that includes this new reference source is Pandas(Index=219264, subject_id='HGNC:11164', subject_label='SNRPN', relation_id='RO:0002200', relation_label='has phenotype', object_id='HP:0000708', object_label='Behavioral abnormality', reference_id_list='MONDO:0008300|NCBIBSgene:pws')
The method will continue to run without problem, writing the CURIE instead of the URI,until the dictionary is updated.
Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSg

Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'
The edge that includes this new reference source is Pandas(Index=361842, subject_id='HGNC:9586', subject_label='PTCH2', relation_id='RO:0002200', relation_label='has phenotype', object_id='HP:0000238', object_label='Hydrocephalus', reference_id_list='MONDO:0007187|MONDO:0007959|NCBIBSgene:bcns')
The method will continue to run without problem, writing the CURIE instead of the URI,until the dictionary is updated.
Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NC

Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'
The edge that includes this new reference source is Pandas(Index=482942, subject_id='HGNC:11164', subject_label='SNRPN', relation_id='RO:0002200', relation_label='has phenotype', object_id='HP:0008872', object_label='Feeding difficulties in infancy', reference_id_list='NCBIBSgene:pws')
The method will continue to run without problem, writing the CURIE instead of the URI,until the dictionary is updated.
Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'


Detected a new reference source in Monarch not yet implemented in this module. The new source should be added to the dictionary of sources.Otherwise, the source CURIE cannot be translated to the corresponding URI.
In the build_edges() method, update 'uriPrefixes_dct' dictionary with 'NCBIBSgene'
The edge that includes this new reference source is Pandas(Index=543024, subject_id='HGNC:583', subject_label='APC', relation_id='RO:0002200', relation_label='has phenotype', object_id='HP:0000164', object_label='Abnormality of the dentition', reference_id_list='PMID:8929955|PMID:14633595|PMID:24735542|PMID:20223039|PMID:17293347|PMID:8395941|PMID:20685668|PMID:9824584|PMID:9101302|PMID:17411426|PMID:27158207|PMID:12173026|PMID:15108286|PMID:9452101|PMID:23159591|PMID:24664542|PMID:11852337|PMID:8162022|PMID:9890479|PMID:25980754|PMID:1651563|PMID:17963004|PMID:28533537|PMID:12007223|PMID:26681312|PMID:1316610|PMID:25590978|PMID:1338764|PMID:20434453|PMID:15459959|PMID:25317407|PMID:8940264|PMI


* This is the size of the nodes file data structure: (18492, 6)
* These are the nodes attributes: Index(['id', 'semantic_groups', 'preflabel', 'name', 'synonyms',
       'description'],
      dtype='object')
* This is the first record:
           id semantic_groups preflabel                   name  \
0  HGNC:23796            GENE    MAGEB6  MAGE family member B6   

                synonyms                                        description  
0  CT3.4|MAGE-B6|MAGEB6A  This gene is a member of the MAGEB gene family...  

The Monarch network nodes are built and saved at: /home/karolis/LUMC/HDSR/bioknowledge-reviewer/bioknowledge_reviewer/monarch/monarch_nodes_v2021-04-12.csv


Finished build_nodes().


The function "build_edges()" is running...

Preparing networks...
Curated:
(1, 9)
Index(['subject_id', 'property_id', 'object_id', 'reference_uri',
       'reference_supporting_text', 'reference_date', 'property_label',
       'property_description', 'property_uri'],
      dtype='object')

In [21]:
reg_graph_edges

NameError: name 'reg_graph_edges' is not defined

In [5]:
edges_df = pd.read_csv("./graph/graph_edges_v2021-04-12_alt_del.csv")
nodes_df = pd.read_csv("./graph/graph_nodes_v2021-04-12.csv")

In [6]:
%%time
# create a Neo4j server instance
neo4j_dir = neo4jlib.create_neo4j_instance('4.2.1')
print('The name of the neo4j directory is {}'.format(neo4j_dir))

# import to graph database
## prepare the graph to neo4j format
#edges_df = utils.get_dataframe(edges)
#nodes_df = utils.get_dataframe(nodes)
statements = neo4jlib.get_statements(edges_df)
concepts = neo4jlib.get_concepts(nodes_df)
print('statements: ', len(statements))
print('concepts: ',len(concepts))

## save files into neo4j import dir
neo4j_path = './{}'.format(neo4j_dir)
neo4jlib.save_neo4j_files(statements, neo4j_path, file_type = 'statements')
neo4jlib.save_neo4j_files(concepts, neo4j_path, file_type = 'concepts')

## import graph into neo4j database
neo4jlib.do_import(neo4j_path)

Creating a Neo4j community v4.2.1 server instance...
Preparing the server...
Configuration adjusted!
Starting the server...
Neo4j v4.2.1 is running.
The name of the neo4j directory is neo4j-community-4.2.1
statements:  593222
concepts:  19721

File './neo4j-community-4.2.1/import/ngly1/ngly1_statements.csv' saved.

File './neo4j-community-4.2.1/import/ngly1/ngly1_concepts.csv' saved.

The function "do_import()" is running...

The graph is imported into the server. Neo4j is running.You can start exploring and querying for hypothesis. If you change ports or authentication in the Neo4j configuration file, the hypothesis-generation modules performance, hypothesis.py and summary.py, will be affected.

CPU times: user 4.22 s, sys: 179 ms, total: 4.4 s
Wall time: 18.5 s


In [ ]:
# print type of objects
print('type edges:', type(statements))
print('type nodes:', type(concepts))
print()

# print objects sizes
print('len edges:', len(statements))
print('len nodes:', len(concepts))
print()

# print object attribute
print('attribute edges:', statements.columns)
print('attribute nodes:', concepts.columns)

## hypothesis-generation library
### Query the graph for mechanistic explanation, then summarize the extracted paths
#### import hypothesis, summary


Tasks:

* Retrieve orthopheno paths with the `query` method.
* Retrieve orthopheno paths using relaxing node degree parameters with the `query` method.
* Retrieve orthopheno paths from a more open query topology with the `open_query` method.
* Get hypothesis summaries

### Ortopheno query with general nodes/relations removed

In [3]:
from neo4j import GraphDatabase
import sys,os
import json
import yaml
import datetime
import neo4j.exceptions

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
def parse_path( path ):
    """
    This function parses neo4j results.
    :param path: neo4j path object
    :return: parsed path dictionary
    """

    out = {}
    out['Nodes'] = []
    for node in path['path'].nodes:
        n = {}
        n['idx'] = node.id
        n['label'] = list(node.labels)[0]
        n['id'] = node.get('id')
        n['preflabel'] = node.get('preflabel')
        n['name'] = node.get('name')
        n['description'] = node.get('description')
        out['Nodes'].append(n)
    out['Edges'] = []
    for edge in path['path'].relationships:
        e = {}
        e['idx'] = edge.id
        e['start_node'] = edge.start_node.id
        e['end_node'] = edge.end_node.id
        e['type'] = edge.type
        e['preflabel'] = edge.get('property_label')
        e['references'] = edge.get('reference_uri')
        out['Edges'].append(e)
    return out

In [ ]:
def get_node(source, target, port='7687'):
    """
    This function checks if a node exists within the graph
    """
    try:
        driver = GraphDatabase.driver("bolt://localhost:{}".format(port), auth=("neo4j", "ngly1"))
    except neo4j.exceptions.ServiceUnavailable:
        raise
    outputAll = list()
    with driver.session() as session:
        query = """MATCH (source { id: '""" + source + """' }), (target { id: '""" + target +  """' }), p = allShortestPaths((source)-[*..15]-(target)) RETURN p"""
        result = session.run(query)
        print(result)
        x = []
        for record in result:
            #path_dct = parse_path(record)
            x.append(record)
        print(x)
        return x, result

In [ ]:
x, result = get_node('HGNC:4851', 'HGNC:18229')

In [ ]:
x[1]

In [ ]:
result.consume()

In [ ]:
def query_shortest_paths(source, target, max_length=4, port='7687'):
    """
    This function gets the shortest paths between source and
    target. max_length determines the maximum path size allowed.
    """
    try:
        driver = GraphDatabase.driver("bolt://localhost:{}".format(port), auth=("neo4j", "ngly1"))
    except neo4j.exceptions.ServiceUnavailable:
        raise
    outputAll = list()
    with driver.session() as session:
        print("kaas")
        query = """MATCH (source { id: '""" + source + """' }), (target { id: '""" + target +  """' }), p = shortestPath((source)-[*..15]-(target)) RETURN p"""
        result = session.run(query)
        print(result)
        return result

In [ ]:
seed = list([
        'HGNC:4851',  # Htt human gene
        'HGNC:18229'  # Rhes human gene
])
result = query_shortest_paths(seed[0], seed[1], max_length=999)

In [ ]:
summary = result.consume()

In [ ]:
summary.query

In [ ]:
result.data()

In [ ]:
%%time
# get orthopheno paths
seed = list([
        'HGNC:4851',  # Htt human gene
        'HGNC:18229'  # Rhes human gene
])
hypothesis.query(seed,queryname='Htt_Rhes',port='7687') 

In [ ]:
%%time
# get orthopheno paths relaxing pathway and phenotype node degrees
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
hypothesis.query(seed, queryname='Htt_Rhes', pwdegree='1000', phdegree='1000', port='7687')

In [ ]:
%%time
# get orthopheno paths from a more open query topogology
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
hypothesis.open_query(seed,queryname='Htt_Rhes',port='7687')

In [ ]:
import hypothesis

In [ ]:
%%time
seed = list([
        'HGNC:4851',  # NGLY1 human gene
        'HGNC:18229'  # AQP1 human gene
])
r = hypothesis.shortest_paths(seed[0], seed[1], max_length=50, port='7687')

In [ ]:
print(r)

In [ ]:
%%time
# get summary
data = summary.path_load('./hypothesis/query_ngly1_aqp1_pwdl1000_phdl1000_paths_v2020-09-14.json')

# parse data for summarization
data_parsed = list()
for query in data:
    query_parsed = summary.query_parser(query)
    data_parsed.append(query_parsed)
summary.metapaths(data_parsed)
summary.nodes(data_parsed)
summary.node_types(data_parsed)
summary.edges(data_parsed)
summary.edge_types(data_parsed)